In [ ]:
#!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.model_selection import train_test_split

In [ ]:
# Variable Pembantu
layer = {
    'input' : 1,
    'output' : 1
}

EPOCH = 1000
batch_size = 2
time_step = 12

In [ ]:
dataset = pd.read_csv('dataset.csv')

In [ ]:
prices = dataset['Price'].values
prices

array([290.41  , 308.14  , 324.66  , 305.35  , 293.26  , 286.41  ,
       312.96  , 340.72  , 375.67  , 338.66  , 344.36  , 348.69  ,
       373.85  , 399.05  , 400.49  , 407.97  , 432.44  , 435.82  ,
       444.83  , 426.28  , 432.92  , 512.54  , 529.1   , 554.04  ,
       583.49  , 607.89  , 565.34  , 526.16  , 559.26  , 577.08  ,
       579.01  , 569.77  , 575.35  , 557.82  , 539.14  , 529.8   ,
       514.45  , 552.72  , 548.49  , 547.87  , 537.37  , 526.02  ,
       631.38  , 627.54  , 603.28  , 715.73  , 748.81  , 769.5   ,
       731.53  , 700.32  , 749.25  , 690.7   , 731.74  , 685.47  ,
       772.71  ,  40.037 ,  40.303 ,  41.237 ,  40.879 ,  40.016 ,
        43.89  ,  44.943 ,  46.165 ,  46.811 ,  47.516 ,  46.5   ,
        49.47  ,  52.32  ,  55.26  ,  57.1577,  56.96  ])

In [ ]:
def get_batch(dataset, time_step, batch_size):
  input_batch = np.zeros(shape=(batch_size, time_step, layer['input']))
  output_batch = np.zeros(shape=(batch_size, time_step, layer['output']))
  
  for i in range(batch_size):
    point = np.random.randint(0, len(dataset) - time_step)
    input_batch += dataset[point : point + time_step]
    output_batch += dataset[point + 1 : point + time_step + 1]

  return input_batch, output_batch

In [ ]:
train_prices, test_prices = train_test_split(prices, test_size=0.17, shuffle=False)

In [ ]:
train_prices = train_prices.reshape(-1, 1)
test_prices = test_prices.reshape(-1, 1)

In [ ]:
scaler = StandardScaler().fit(train_prices)
train_prices = scaler.transform(train_prices)
test_prices = scaler.transform(test_prices)

In [ ]:
timestep = 12

In [ ]:
train_prices.shape

(58, 1)

In [ ]:
def construct_dataset(in_prices):
  features = []
  targets = []

  for i in range(timestep, in_prices.shape[0]):
    features.append(
        in_prices[i - timestep : i]
    )
    targets.append(
        in_prices[i]
    )
  features = np.array(features, 'float32')
  targets = np.array(targets, 'float32')

  return features, targets

In [ ]:
x_train, y_train = construct_dataset(train_prices)
x_test, y_test = construct_dataset(test_prices)

In [ ]:
input_placeholder = tf.placeholder(tf.float32, [None, timestep, 1])
target_placeholder = tf.placeholder(tf.float32, [None, 1])

In [ ]:
cell = SimpleRNNCell(12, activation='relu')

rnn_layer = RNN(cell, dtype=tf.float32)
output_layer = Dense(1)

In [ ]:
out_tensor = rnn_layer(input_placeholder)
out_tensor = output_layer(out_tensor)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# mse
loss_tensor = tf.reduce_mean(0.5 * (out_tensor - target_placeholder)**2)

# optimal rmse (root mean squared error)
rmse_tensor = tf.reduce_mean(tf.sqrt(0.5 * (out_tensor - target_placeholder)**2))

In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(loss_tensor)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
saver = tf.train.Saver()

In [ ]:
num_epoch = 5000

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  train_dict = {
      input_placeholder:x_train,
      target_placeholder:y_train
  }
  test_dict = {
      input_placeholder:x_test,
      target_placeholder:y_test
  }

  best_loss = float('inf')
  for epoch in range(0,num_epoch):
    sess.run(optimizer, feed_dict=train_dict)

    loss = sess.run(loss_tensor, feed_dict=train_dict)
    val_loss = sess.run(loss_tensor, feed_dict=test_dict)

    rmse = sess.run(rmse_tensor, feed_dict=train_dict)
    val_rmse = sess.run(rmse_tensor, feed_dict=test_dict)

    if epoch % 1000:
      saver.save(sess, './best_model.ckpt')
      
    if epoch % 250 == 0:
      print(f'Epoch {epoch} - loss {loss:.4f} ')

Epoch 0 - loss 0.8987 
Epoch 250 - loss 0.1878 
Epoch 500 - loss 0.1802 
Epoch 750 - loss 0.1786 
Epoch 1000 - loss 0.1773 
Epoch 1250 - loss 0.1754 
Epoch 1500 - loss 0.0230 
Epoch 1750 - loss 0.0201 
Epoch 2000 - loss 0.0192 
Epoch 2250 - loss 0.0180 
Epoch 2500 - loss 0.0166 
Epoch 2750 - loss 0.0149 
Epoch 3000 - loss 0.0124 
Epoch 3250 - loss 0.0095 
Epoch 3500 - loss 0.0070 
Epoch 3750 - loss 0.0047 
Epoch 4000 - loss 0.0037 
Epoch 4250 - loss 0.0032 
Epoch 4500 - loss 0.0026 
Epoch 4750 - loss 0.0021 


In [ ]:
x = prices[-12:]
x = x.reshape(-1, 1)
x = scaler.transform(x)
x = np.array([x])

In [ ]:
with tf.Session() as sess:
  saver.restore(sess, './best_model.ckpt')

  pred = sess.run(out_tensor, feed_dict={input_placeholder:x})
  pred = scaler.inverse_transform(pred)
  print(f'predicted stock price of google in next month: {pred}')

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
predicted stock price of google in next month: [[348.59103]]
